# Error Analysis

In [5]:
import pandas as pd
import spacy
import thesis_utils
from IPython.display import display, HTML
from collections import defaultdict
from tqdm import tqdm
import random
import os
import csv

/home/elisabetta/anaconda3/lib/python3.12/site-packages/torch/cuda/__init__.py:734: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
/home/elisabetta/anaconda3/lib/python3.12/site-packages/torch/cuda/__init__.py:734: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
comet_ml is installed but `COMET_API_KEY` is not set.


In [6]:
xlm_roberta_merlin = './predictions_on_processedfiles/proc_ita_xlmroberta.tsv'

In [7]:
xlm_roberta_merlin_sents = thesis_utils.read_tsv_file_and_find_sentences_with_headers(xlm_roberta_merlin)

In [8]:
def evaluate_error_types_recall_only_merlin(sentences, dataset, print_results=True):
    error_stats = defaultdict(lambda: {"gold_i": 0, "correct_pred_i": 0})
    
    for sentence in sentences:
        for line in sentence:
            if dataset == 'merlin':
                error_type = line[6]
                gold_label = line[-2]
                pred_label = line[-1]
                if gold_label == 'i':
                    error_stats[error_type]["gold_i"] += 1
                    if pred_label == 'i':
                        error_stats[error_type]["correct_pred_i"] += 1
            elif dataset == 'fce':
                error_type = line[3]
                gold_label = line[-2]
                pred_label = line[-1]
                if gold_label == 'i':
                    error_stats[error_type]["gold_i"] += 1
                    if pred_label == 'i':
                        error_stats[error_type]["correct_pred_i"] += 1
            elif dataset == 'realec':
                gold_label = line[-2]
                pred_label = line[-1]
                error_type = line[2]#they looked only at the first layer of the annotations
                # for i in range(2, len(line) - 2, 2):
                #     error_type = line[i]
                if gold_label == 'i':
                    error_stats[error_type]["gold_i"] += 1
                    if pred_label == 'i':
                        error_stats[error_type]["correct_pred_i"] += 1

    if print_results:
        header = f"| {'Error Code':<23} | {'Gold':>6} | {'True Pos.':>7} | {'False Neg.':>10} | {'Recall':>6} |"
        print(header)
        print("|" + "-"*(len(header)-2) + "|")

        total_gold = 0
        total_correct = 0

        sorted_items = sorted(
            error_stats.items(), 
            key=lambda item: item[1]["correct_pred_i"] / item[1]["gold_i"] if item[1]["gold_i"] > 0 else 0.0, 
            reverse=True
        )

        for error_type, stats in sorted_items:
            gold_i = stats["gold_i"]
            correct_pred_i = stats["correct_pred_i"]
            false_neg = gold_i - correct_pred_i
            recall = correct_pred_i / gold_i if gold_i > 0 else 0.0
            print(f"| {error_type:<23} | {gold_i:6d} | {correct_pred_i:9d} | {false_neg:10d} | {recall:6.2f} |")
            
            total_gold += gold_i
            total_correct += correct_pred_i

        total_false_neg = total_gold - total_correct
        overall_recall = total_correct / total_gold if total_gold > 0 else 0.0

        print("|" + "-"*(len(header)-2) + "|")
        print(f"| {'TOTAL':<23} | {total_gold:6d} | {total_correct:9d} | {total_false_neg:10d} | {overall_recall:6.2f} |")

    return error_stats


In [9]:
roberta_merlin_stats = evaluate_error_types_recall_only_merlin(xlm_roberta_merlin_sents,'merlin')

| Error Code              |   Gold | True Pos. | False Neg. | Recall |
|--------------------------------------------------------------------|
| C_Coh_jump              |      1 |         1 |          0 |   1.00 |
| O_Capit                 |     43 |        36 |          7 |   0.84 |
| G_Refl                  |      4 |         3 |          1 |   0.75 |
| O_Graph                 |    154 |       108 |         46 |   0.70 |
| O_Wordbd                |     10 |         7 |          3 |   0.70 |
| S_Var                   |      3 |         2 |          1 |   0.67 |
| G_Verb_compl            |     17 |        11 |          6 |   0.65 |
| O_Punct                 |     62 |        38 |         24 |   0.61 |
| G_Prep                  |     82 |        49 |         33 |   0.60 |
| G_Verb_main             |     28 |        16 |         12 |   0.57 |
| G_Morphol_Wrong         |     52 |        29 |         23 |   0.56 |
|                         |    295 |       162 |        133 |   0.55 |
| G_In

In [10]:
def find_false_positives_and_false_negatives(list_sentences, dataset):
    only_fp = []
    only_fn = []
    both_fp_fn = []

    false_negatives_by_type_clean = defaultdict(set)
    false_negatives_by_type_all = defaultdict(set)

    for sentence in list_sentences:
        has_fp = any(token[-1] == 'i' and token[-2] == 'c' for token in sentence)
        has_fn = any(token[-1] == 'c' and token[-2] == 'i' for token in sentence)

        if has_fp and not has_fn:
            only_fp.append(sentence)
        elif has_fn and not has_fp:
            only_fn.append(sentence)
        elif has_fp and has_fn:
            both_fp_fn.append(sentence)

    for sentence in only_fn:
        if dataset == 'merlin':
            for token in sentence:
                if token[-1] == 'c' and token[-2] == 'i' and len(token) > 6:
                    error_type = token[6] if token[6] else 'NO_LABEL'
                    false_negatives_by_type_clean[error_type].add(tuple(tuple(t) for t in sentence))
        elif dataset == 'fce':
            for token in sentence:
                if token[-1] == 'c' and token[-2] == 'i' and len(token) > 3:
                    error_type = token[3] if token[3] else 'NO_LABEL'
                    false_negatives_by_type_clean[error_type].add(tuple(tuple(t) for t in sentence))
        else:
            for token in sentence:
                if token[-1] == 'c' and token[-2] == 'i':
                    error_type = token[2] if token[2] else 'NO_LABEL'
                    false_negatives_by_type_clean[error_type].add(tuple(tuple(t) for t in sentence))

    for sentence in both_fp_fn:
        if dataset == 'merlin':
            for token in sentence:
                if token[-1] == 'c' and token[-2] == 'i' and len(token) > 6:
                    error_type = token[6] if token[6] else 'NO_LABEL'
                    false_negatives_by_type_all[error_type].add(tuple(tuple(t) for t in sentence))
        elif dataset == 'fce':
            for token in sentence:
                if token[-1] == 'c' and token[-2] == 'i' and len(token) > 3:
                    error_type = token[3] if token[3] else 'NO_LABEL'
                    false_negatives_by_type_all[error_type].add(tuple(tuple(t) for t in sentence))
        else:
            for token in sentence:
                if token[-1] == 'c' and token[-2] == 'i':
                    error_type = token[2] if token[2] else 'NO_LABEL'
                    false_negatives_by_type_all[error_type].add(tuple(tuple(t) for t in sentence))

    false_negatives_by_type_clean = {k: list(v) for k, v in false_negatives_by_type_clean.items()}
    false_negatives_by_type_all = {k: list(v) for k, v in false_negatives_by_type_all.items()}

    return only_fp, only_fn, both_fp_fn, false_negatives_by_type_clean, false_negatives_by_type_all


roberta_merlin_only_fp, roberta_merlin_only_fn, roberta_merlin_fp_and_fn, roberta_merlin_only_fn_error_type, roberta_merlin_fn_fp_error_type = find_false_positives_and_false_negatives(xlm_roberta_merlin_sents,'merlin')
print(f"False Positive Sentences (roberta-merlin): {len(roberta_merlin_only_fp)}")
print(f"False Negative Sentences (roberta-merlin): {len(roberta_merlin_only_fn)}\n")

False Positive Sentences (roberta-merlin): 42
False Negative Sentences (roberta-merlin): 260



In [11]:
for key in sorted(set(roberta_merlin_only_fn_error_type) | set(roberta_merlin_fn_fp_error_type)):
    fn_count = len(roberta_merlin_only_fn_error_type.get(key, []))
    fp_fn_count = len(roberta_merlin_fn_fp_error_type.get(key, []))
    print(f"| {key:<18} | FN: {fn_count:6d} | FP+FN: {fp_fn_count:6d} |")

| C_Con_accur        | FN:      1 | FP+FN:      0 |
| G_Agr              | FN:      8 | FP+FN:      1 |
| G_Art              | FN:     28 | FP+FN:      8 |
| G_Clit             | FN:      7 | FP+FN:      0 |
| G_Conj             | FN:     13 | FP+FN:      4 |
| G_Inflect_Inexist  | FN:     10 | FP+FN:      2 |
| G_Morphol_Wrong    | FN:     15 | FP+FN:      5 |
| G_Neg              | FN:      2 | FP+FN:      0 |
| G_Pos              | FN:     10 | FP+FN:      2 |
| G_Prep             | FN:     27 | FP+FN:      4 |
| G_Refl             | FN:      1 | FP+FN:      0 |
| G_Valency          | FN:     12 | FP+FN:      5 |
| G_Verb             | FN:     20 | FP+FN:      5 |
| G_Verb_compl       | FN:      6 | FP+FN:      0 |
| G_Verb_main        | FN:      8 | FP+FN:      1 |
| G_Wo               | FN:     16 | FP+FN:      1 |
| H_Intellts         | FN:      1 | FP+FN:      0 |
| H_Intelltxt        | FN:     21 | FP+FN:     10 |
| NO_LABEL           | FN:     80 | FP+FN:     22 |
| O_Apostr  

In [74]:
def visualize_aligned_tokens_aligned(data):
    for i, sentence in enumerate(data):
        print(f"\n=== Sentence {i+1} ===")

        # Extract info
        tokens = [tok[1] for tok in sentence]
        corrections = [tok[2] for tok in sentence]
        corrections2 = [tok[4] for tok in sentence]
        error_types = [tok[6] if tok[6] else "-" for tok in sentence]
        golds = [tok[7] for tok in sentence]
        preds = [tok[8] for tok in sentence]

        col_widths = [max(len(tok), len(corr), len(err), len(g), len(p))
                      for tok, corr, err, g, p in zip(tokens, corrections, error_types, golds, preds)]
        padded = lambda row: " | ".join(f"{item:<{w}}" for item, w in zip(row, col_widths))

        print("TOKENS:       ", padded(tokens))
        print("CORR. TH1:  ", padded(corrections))
        print("CORR. TH2:  ", padded(corrections2))
        print("ERROR TYPES:  ", padded(error_types))
        print("GOLD LABELS:  ", padded(golds))
        print("PRED LABELS:  ", padded(preds))




visualize_aligned_tokens_aligned(roberta_merlin_only_fp)


=== Sentence 1 ===
TOKENS:        Ciao  | Caro  | ,     | come  | stai  | ?    
CORR. TH1:   Ciao  | Caro  | ,     | come  | stai  | ?    
CORR. TH2:   Ciao  | caro  | ,     | come  | stai  | ?    
ERROR TYPES:   S_Txt | S_Txt | S_Txt | S_Txt | S_Txt | S_Txt
GOLD LABELS:   c     | c     | c     | c     | c     | c    
PRED LABELS:   c     | i     | c     | c     | c     | c    

=== Sentence 2 ===
TOKENS:        Tanto       | non         | bisognò     | di          | prenotare   | un          | alloggio    | .        
CORR. TH1:   Tanto       | non         | bisogno     | di          | prenotare   | un          | alloggio    | .        
CORR. TH2:               |             |             |             |             |             |             |          
ERROR TYPES:   G_Verb_main | G_Verb_main | G_Verb_main | G_Verb_main | G_Verb_main | G_Verb_main | G_Verb_main | G_Valency
GOLD LABELS:   c           | c           | i           | c           | c           | c           | c          

In [27]:
# def save_sentences_to_tsv(data, error_type, file_path):
#     mode = 'a' if os.path.exists(file_path) else 'w'

#     with open(file_path, mode, encoding='utf-8', newline='') as f:
#         writer = csv.writer(f, delimiter='\t')

#         if mode == 'w':
#             writer.writerow(["error_type", "sentence"])

#         for sentence in data:
#             tokens = [tok[1] for tok in sentence]
#             sentence_text = " ".join(tokens)
#             writer.writerow([error_type, sentence_text])

In [79]:
# save_sentences_to_tsv(roberta_merlin_only_fp, 'FP', 'classified_sentences_merlin_FP.tsv')

# Qwen!!!

In [11]:
from openai import OpenAI
from collections import defaultdict as dd
from openai import APITimeoutError

client = OpenAI(base_url="http://localhost:8000/v1", api_key="cltl")

##  Prompt Mode

In [22]:
def query_LLM(model_client, prompt, temp=0.6):
    completion = model_client.chat.completions.create(
        model="local-model",
        messages=[{"role": "system", "content": prompt}],
        temperature=temp,
        stream=False,
    )

    return completion.choices[0].message.content

In [24]:
def get_corrections_tokenized_with_labels_from_data(sentence, model_client, max_attempts=2):
    tokens = [tok[1] for tok in sentence]
    sentence_str = " ".join(tokens)

    gold_correction = []
    error_types = []

    for tok in sentence:
        gold = tok[-2]
        pred = tok[-1]
        orig = tok[1]
        corr = tok[2]
        etype = tok[6] if tok[6] else "NO_LABEL"

        if gold == 'i':
            error_types.append(etype)
            if corr:
                gold_correction.append(corr)
            else: 
                continue
        else:
            gold_correction.append(orig)

    prompt = f"""You are a careful Italian language editor.

Given a tokenized sentence, return exactly five corrected versions. Each version should:
- Be minimally corrected (fix grammar and lexical errors only)
- Be tokenized (space-separated words)
- Be written on a separate line
- Use deletions if a word is clearly unnecessary or incorrect
- Do not add explanations
- If fewer than five unique corrections are possible, write an empty line for each missing correction

Input: {sentence_str}
Output:
"""

    for attempt in range(max_attempts):
        response = query_LLM(model_client, prompt)

        if response:
            lines = [line.strip() for line in response.strip().split("\n")]
            valid_corrections = [line.split() for line in lines if line.strip()]

            if valid_corrections:
                labels = []
                for corrected in valid_corrections:
                    label_seq = ["c" if i < len(tokens) and orig == corr else "i"
                                 for i, (orig, corr) in enumerate(zip(tokens, corrected))]
                    labels.append(label_seq)

                gold_labels = [tok[-2] for tok in sentence]
                pred_labels = [tok[-1] for tok in sentence]
                
                print("\nOriginal:    ", sentence_str)
                print("Gold labels: ", " ".join(gold_labels))
                print("Pred labels: ", " ".join(pred_labels))
                print("Gold:        ", " ".join(gold_correction))
                for idx, corr in enumerate(valid_corrections):
                    print(f"Correction {idx+1}:", " ".join(corr))
                print("Error types: ", list(set(error_types)))
                print()

                return {
                    "original": tokens,
                    "gold_correction": gold_correction,
                    "error_types": list(set(error_types)),
                    "llm_corrections": valid_corrections,
                    "labels": labels
                }
            else:
                print(f"Attempt {attempt+1}: No valid corrections, retrying...")
        else:
            print(f"Attempt {attempt+1}: Empty response, retrying...")

    print("\nOriginal:", sentence_str)
    print("No corrections found.\n")
    return {
        "original": tokens,
        "gold_correction": gold_correction,
        "error_types": list(set(error_types)),
        "llm_corrections": [],
        "labels": []
    }

# GENERATION BY ERROR TYPE - MERLIN - XLM-ROBERTA

## False Negatives Only

### ROBERTA --> G_Wo

In [29]:
all_G_Wo_roberta_corrections_FN = []

for sentence in tqdm(roberta_merlin_only_fn_error_type['G_Wo']):
    sentence_corrections = get_corrections_tokenized_with_labels_from_data(sentence, client)
    all_G_Wo_roberta_corrections_FN.append(sentence_corrections)

  6%|██▊                                         | 1/16 [00:17<04:17, 17.15s/it]


Original:     Ma non c' è così tutto che abbiamo pensato .
Gold labels:  i c i c i c i c c c
Pred labels:  c c c c c c c c c c
Gold:         ma non è tutto come abbiamo pensato .
Correction 1: Ma non ce ne è così tanto che abbiamo pensato .
Correction 2: Ma non c'è così tanto che abbiamo pensato .
Correction 3: Ma non c'è così tutto che abbiamo pensato .
Correction 4: Ma non ci è così tutto che abbiamo pensato .
Correction 5: Ma non c'è così completo che abbiamo pensato .
Error types:  ['G_Wo']



 12%|█████▌                                      | 2/16 [00:41<04:57, 21.28s/it]


Original:     È molto importante di avere un lavoro fisso e per trovarlo ti vorrei aiutare .
Gold labels:  c c c i c c c c c i i c c c i
Pred labels:  c c c i c c c c c c c c c c c
Gold:         È molto importante avere un lavoro fisso e ti vorrei aiutare .
Correction 1: È molto importante avere un lavoro fisso e per trovarlo ti vorrei aiutare.
Correction 2: È molto importante avere un lavoro fisso e per trovarlo mi vorresti aiutare.
Correction 3: È molto importante avere un lavoro fisso e per trovarlo ti vogliero' aiutare.
Correction 4: È molto importante avere un lavoro fisso e per trovarlo ti vorrei aiutarti.
Correction 5: È molto importante avere un lavoro fisso e per trovarlo ti vorrei aiutare .
Error types:  ['G_Prep', 'G_Wo']



 19%|████████▎                                   | 3/16 [01:04<04:45, 21.98s/it]

...


Original:     La settimana scorsa hanno comminciato cinque nuove persone !
Gold labels:  c c c c i c i i c
Pred labels:  c c c c i c c c c
Gold:         La settimana scorsa hanno cominciato cinque nuove !
Correction 1: La settimana scorsa hanno cominciato cinque nuove persone !
Correction 2: La settimana scorsa sono commesse cinque nuove persone !
Correction 3: La settimana scorsa cinque persone si sono commesse !
Correction 4: La settimana scorsa cinque nuove persone hanno cominciato !
Correction 5: La settimana scorsa cinque nuove persone hanno iniziato !
Error types:  ['G_Wo']



 94%|████████████████████████████████████████▎  | 15/16 [08:04<00:42, 42.06s/it]


Original:     Ancora volta mi dispiace , ma non è mia colpa , Mio capo ha organizzato una runione giovedi pomeriggio e per questo dobbiamo cambiare nostro appuntamento .
Gold labels:  c i c c c c c c i c i i c c c c i i c c c c c c i c c
Pred labels:  c c c c c c c c c c c i c c c c i i c c c c c c i c c
Gold:         Ancora volta mi dispiace , ma non è colpa : mio capo ha organizzato una riunione giovedì pomeriggio e per questo dobbiamo cambiare nostro appuntamento .
Correction 1: Ancora volta mi dispiace, ma non è mia colpa, Mio capo ha organizzato una riunione giovedì pomeriggio e per questo dobbiamo cambiare nostro appuntamento.
Correction 2: Ancora volta mi dispiace, ma non è mia colpa, Mio capo ha organizzato una riunione giovedì pomeriggio e per questo dobbiamo cambiarlo appuntamento.
Correction 3: Ancora volta mi dispiace, ma non è mia colpa, Mio capo ha organizzato una riunione giovedì pomeriggio e per questo devo cambiare l'appuntamento.
Correction 4: Ancora volta mi dispiac

100%|███████████████████████████████████████████| 16/16 [08:56<00:00, 33.56s/it]


Original:     Invece , l' inglese , che lo aveva usato prima moltissimo quando lavorava , solo l' uso quando stiamo con gli amici stranieri che non parlano italiano .
Gold labels:  c i c c c c i i c i c i i c i i i c c c c c c c c c c c
Pred labels:  c c c c c c c c c c c c c c c c c c c c c c c c c c c c
Gold:         Invece l' inglese , che ho usato moltissimo quando lavoravo , solo quando stiamo con gli amici stranieri che non parlano italiano .
Correction 1: Invece, l'inglese, che lo aveva usato molto prima quando lavorava, solo l'uso quando stiamo con gli amici stranieri che non parlano italiano .
Correction 2: Invece, l'inglese, che lo aveva usato moltissimo prima quando lavorava, l'uso solo quando stiamo con gli amici stranieri che non parlano italiano .
Correction 3: Invece, l'inglese, che lo aveva usato molto prima quando lavorava, l'uso solo quando stiamo con gli amici stranieri che non parlano italiano .
Correction 4: Invece, l'inglese, che lo aveva usato moltissimo prima q

### ROBERTA --> G_Verb

In [32]:
all_G_Verb_roberta_corrections_FN = []

for sentence in tqdm(roberta_merlin_only_fn_error_type['G_Verb']):
    sentence_corrections = get_corrections_tokenized_with_labels_from_data(sentence, client)
    all_G_Verb_roberta_corrections_FN.append(sentence_corrections)

  5%|██▏                                         | 1/20 [00:34<11:01, 34.83s/it]


Original:     Puoi dirmi dove hai lavorato adesso ?
Gold labels:  c c c i i c c
Pred labels:  c c c c c c c
Gold:         Puoi dirmi dove lavori adesso ?
Correction 1: Puoi dirmi dove hai lavorato recently?
Correction 2: Puoi dirmi dove lavori adesso?
Correction 3: Puoi dirmi dove hai lavorato di recente?
Correction 4: Puoi dirmi dove sei impiegato adesso?
Correction 5: Puoi dirmi dove hai lavorato ultimamente?
Correction 6: (Note: The fifth line is left empty as there are only four unique corrections possible while adhering to the constraints.)
Error types:  ['G_Verb']



 10%|████▍                                       | 2/20 [01:06<09:53, 32.98s/it]


Original:     Ma la prossima settimana sara al IKEA e lì trovo un regalo buono .
Gold labels:  c c c c i i c c c i c c c c
Pred labels:  c c c c i i c c c c c c c c
Gold:         Ma la prossima settimana sarò all' IKEA e lì troverò un regalo buono .
Correction 1: Ma la prossima settimana sarò al IKEA e lì trovo un regalo buono .
Correction 2: Ma la prossima settimana sarò al IKEA e lí trovo un regalo buono .
Correction 3: Ma la prossima settimana sarò al IKEA e lì trovo un regalo migliore .
Correction 4: Ma la prossima settimana sarò al IKEA e lì trovo un bel regalo .
Correction 5: Ma la prossima settimana sarò al IKEA e lí trovo un bel regalo .
Error types:  ['G_Verb', 'G_Morphol_Wrong', 'O_Graph']



 15%|██████▌                                     | 3/20 [01:37<09:08, 32.28s/it]

...


Original:     Cosi avevamo sentito male .
Gold labels:  i i c c c
Pred labels:  i c c c c
Gold:         così abbiamo sentito male .
Correction 1: Così avevamo sentito male.
Correction 2: Così avevamo sentito male.
Correction 3: Così abbiamo sentito male.
Correction 4: Così avevamo sentito male.
Correction 5: Così avevamo sentito corretto.
Error types:  ['G_Verb', 'O_Graph']



 95%|████████████████████████████████████████▊  | 19/20 [09:13<00:22, 22.94s/it]


Original:     Sempre mi piaceva l' Italia e la pronuncia italiana .
Gold labels:  i i i i i c i c c c
Pred labels:  c c c c c c c c c c
Gold:         sempre piaciuta e la pronuncia italiana .
Correction 1: Sempre mi piaceva l'Italia e la pronuncia italiana .
Correction 2: Sempre mi piace l'Italia e la pronuncia italiana .
Correction 3: Sempre mi piaceva l'Italia e la pronunzia italiana .
Correction 4: Sempre mi piacevo l'Italia e la pronunzia italiana .
Correction 5: Sempre mi piacevo l'Italia e la pronuncia italiana .
Error types:  ['G_Wo', 'G_Verb']



100%|███████████████████████████████████████████| 20/20 [09:34<00:00, 28.70s/it]


Original:     Io lo sapevo , che tu ce la fai senza problemi .
Gold labels:  c c c i c c c c i c c c
Pred labels:  c c c i c c c c c c c c
Gold:         Io lo sapevo che tu ce la facevi senza problemi .
Correction 1: Io lo sapevo che tu ce la fai senza problemi .
Correction 2: Io lo sapevo, tu ci la fai senza problemi.
Correction 3: Io lo sapevo, tu ce la fai senzaproblemi.
Correction 4: Io lo sapevo, tu ce la fai.
Correction 5: Questa frase non permette altre corrette varianti con esattamente cinque modifiche minimali.
Error types:  ['G_Verb', 'O_Punct']



### ROBERTA --> G_Art

In [34]:
all_G_Art_roberta_corrections_FN = []

for sentence in tqdm(roberta_merlin_only_fn_error_type['G_Art']):
    sentence_corrections = get_corrections_tokenized_with_labels_from_data(sentence, client)
    all_G_Art_roberta_corrections_FN.append(sentence_corrections)

  4%|█▌                                          | 1/28 [00:32<14:48, 32.90s/it]


Original:     perchè non parli un po' di matrimonio quando telefoni con me ?
Gold labels:  i c c c c i c c i i i c
Pred labels:  i c c c c c c c c c c c
Gold:         perché non parli un po' del matrimonio quando telefoni ?
Correction 1: perché non parli un po' di matrimonio quando telefoni con me
Correction 2: perché non parli un po' di matrimonio quando chiami con me
Correction 3: perché non parli un po' di matrimonio quando parli con me
Correction 4: perché non parli un po' di matrimonio quando mi telefoni
Correction 5: perché non parli un po' di matrimonio quando mi chiami
Error types:  ['G_Clit', 'O_Graph', 'G_Prep', 'G_Art', 'NO_LABEL']



  7%|███▏                                        | 2/28 [00:39<07:28, 17.26s/it]


Original:     Qualce volta , vado in calcio .
Gold labels:  i c i c i c c
Pred labels:  i c i c c c c
Gold:         Qualche volta vado al calcio .
Correction 1: Qualche volta, vado in calcio .
Error types:  ['G_Art', 'O_Graph', 'O_Punct']



 11%|████▋                                       | 3/28 [01:01<08:12, 19.70s/it]

...


Original:     Mia figlia ha comminciato a giocare al tennis e adesso andiamo a giocare insieme ogni lunedì .
Gold labels:  c c c i c c i c c c c c c c c c c
Pred labels:  c c c i c c c c c c c c c c c c c
Gold:         Mia figlia ha cominciato a giocare a tennis e adesso andiamo a giocare insieme ogni lunedì .
Correction 1: Mia figlia ha cominciato a giocare al tennis e adesso andiamo a giocare insieme ogni lunedì .
Correction 2: Mia figlia ha cominciato a giocare al tennis e adesso lui andiamo a giocare insieme ogni lunedì .
Correction 3: Mia figlia ha cominciato a giocare al tennis e adesso andiamo a giocare insieme un lunedì .
Correction 4: Mia figlia ha cominciato a giocare al tennis e adesso andiamo a giocare insieme ogni lunedì .
Correction 5: Mia figlia ha cominciato a giocare al tennis e adesso andiamo a giocare insieme ogni lunedì .
Error types:  ['G_Art', 'O_Graph']



 96%|█████████████████████████████████████████▍ | 27/28 [13:28<00:26, 26.90s/it]


Original:     E come stanno tua sorella e tuoi genitori ?
Gold labels:  c c c c c c i c c
Pred labels:  c c c c c c c c c
Gold:         E come stanno tua sorella e tuoi genitori ?
Correction 1: E come stanno tua sorella e i tuoi genitori ?
Correction 2: E come stanno tua sorella e tuoi genitori?
Correction 3: E come stanno tua sorella e i tuoi genitori
Correction 4: E come sta tua sorella e tuoi genitori?
Correction 5: E come stanno tua sorella e tuoi genitori?
Error types:  ['G_Art']



100%|███████████████████████████████████████████| 28/28 [14:39<00:00, 31.41s/it]


Original:     Inoltre , come brasiliana e sapendo che il progetto aiuta a finanziare grupi con scopo di sviluppamento in America latina , mi sento ancora più felice e soddisfata di poter partecipare .
Gold labels:  c c c c c c c c c c c c i c i c i c c c c c c c c c c i c c c c
Pred labels:  c c c c c c c c c c c c i c c c i c c c c c c c c c c c c c c c
Gold:         Inoltre , come brasiliana e sapendo che il progetto aiuta a finanziare gruppi con scopo di sviluppamento in America latina , mi sento ancora più felice e soddisfatta di poter partecipare .
Correction 1: Inoltre, come brasiliana e sapendo che il progetto aiuta a finanziare grupi con scopo di sviluppamento in America Latina, mi sento ancora più felice e soddisfatta di poter partecipare.
Correction 2: Inoltre, come brasiliana e sapendo che il progetto aiuta a finanziare grupi con lo scopo di sviluppamento in America Latina, mi sento ancora più felice e soddisfatta di poter partecipare.
Correction 3: Inoltre, come brasiliana

### ROBERTA --> G_Valency

In [36]:
all_G_Valency_roberta_corrections_FN = []

for sentence in tqdm(roberta_merlin_only_fn_error_type['G_Valency']):
    sentence_corrections = get_corrections_tokenized_with_labels_from_data(sentence, client)
    all_G_Valency_roberta_corrections_FN.append(sentence_corrections)

  8%|███▋                                        | 1/12 [00:24<04:30, 24.55s/it]


Original:     Non sto bene è una settimana orribile .
Gold labels:  i c c i c c c c
Pred labels:  c c c i c c c c
Gold:         non sto bene è una settimana orribile .
Correction 1: Non sto bene. È una settimana orribile.
Correction 2: Non sto bene, è stata una settimana orribile.
Correction 3: Non sto bene: è una settimana orribile.
Correction 4: Non sono bene, è una settimana orribile.
Correction 5: Non sto bene, è una settimana orribile .
Error types:  ['G_Valency', 'NO_LABEL']



 17%|███████▎                                    | 2/12 [00:55<04:41, 28.10s/it]


Original:     Secondo me i tuoi amici perdonano e non ti chiedono più perchè non ti faccio più sentire .
Gold labels:  c c c c c i c c c c c i c c i c c c
Pred labels:  c c c c c c c c c c c i c c c c c c
Gold:         Secondo me i tuoi amici perdonano e non ti chiedono più perché non ti fai più sentire .
Correction 1: Secondo me i tuoi amici perdonano e non ti chiedono più perché non ti faccio più sentire .
Correction 2: Secondo me i tuoi amici perdonano e non ti chiedono più il motivo per cui non ti faccio più sentire .
Correction 3: Secondo me i tuoi amici perdonano e non ti chiedono più la ragione per cui non ti faccio più sentire .
Correction 4: Secondo me i tuoi amici perdonano e non ti chiedono più il motivo di non ti faccio più sentire .
Correction 5: Secondo me i tuoi amici perdonano e non ti chiedono più .
Error types:  ['O_Graph', 'G_Agr', 'G_Valency']



 25%|███████████                                 | 3/12 [01:10<03:20, 22.24s/it]

...


Original:     Città X , 24.03.12 Caro Francesco , all' inizio ringrazio per la tua lettera .
Gold labels:  c c c c c c c c c i c c c c c
Pred labels:  c c c c c c c c c c c c c c c
Gold:         Città X , 24.03.12 Caro Francesco , all' inizio ringrazio per la tua lettera .
Correction 1: Città X , 24.03.12 Caro Francesco , all'inizio ringrazio per la tua lettera .
Correction 2: Città X , 24.03.12 Caro Francesco , all'inizio ti ringrazio per la tua lettera .
Correction 3: Città X , 24.03.12 Caro Francesco , al principio ringrazio per la tua lettera .
Correction 4: Città X , 24.03.12 Caro Francesco , al principio ti ringrazio per la tua lettera .
Correction 5: Città X 24.03.12 Caro Francesco all'inizio ringrazio per la tua lettera .
Error types:  ['G_Valency']



 92%|███████████████████████████████████████▍   | 11/12 [05:06<00:30, 30.17s/it]


Original:     Per questo grande passo vi auguro profondamente .
Gold labels:  c c c c c c c i
Pred labels:  c c c c c c c c
Gold:         Per questo grande passo vi auguro profondamente .
Correction 1: Per questo grande passo vi auguro profondamente .
Correction 2: Per questo grande passo vi auguro sinceramente .
Correction 3: Per questo grande passo vi auguro calorosamente .
Correction 4: Per questo grande passo vi auguro terna .
Correction 5: Per questo grande passo vi auguro .
Error types:  ['G_Valency']



100%|███████████████████████████████████████████| 12/12 [06:11<00:00, 30.93s/it]


Original:     Invece , l' inglese , che lo aveva usato prima moltissimo quando lavorava , solo l' uso quando stiamo con gli amici stranieri che non parlano italiano .
Gold labels:  c i c c c c i i c i c i i c i i i c c c c c c c c c c c
Pred labels:  c c c c c c c c c c c c c c c c c c c c c c c c c c c c
Gold:         Invece l' inglese , che ho usato moltissimo quando lavoravo , solo quando stiamo con gli amici stranieri che non parlano italiano .
Correction 1: Invece, l'inglese, che lo aveva usato molto prima quando lavorava, lo usiamo solo quando stiamo con gli amici stranieri che non parlano italiano .
Correction 2: Invece, l'inglese, che lo aveva usato molto prima quando lavorava, solo l'uso quando stiamo con gli amici stranieri che non parlano italiano .
Correction 3: Invece, l'inglese, che lo aveva usato moltissimo prima quando lavorava, lo usiamo solo quando stiamo con gli amici stranieri che non parlano italiano .
Correction 4: Invece, l'inglese, che lo aveva usato molto prim

### ROBERTA --> G_Agr

In [38]:
all_G_Agr_roberta_corrections_FN = []

for sentence in tqdm(roberta_merlin_only_fn_error_type['G_Agr']):
    sentence_corrections = get_corrections_tokenized_with_labels_from_data(sentence, client)
    all_G_Agr_roberta_corrections_FN.append(sentence_corrections)

 12%|█████▋                                       | 1/8 [00:40<04:46, 40.93s/it]


Original:     Mi immaginerei un corso in un gruppo nuomente piccolo - 4 persone al massimo - se questo non è possibile prenderai dei lezioni individuali .
Gold labels:  c c c c c c c i c c c c c c c c c c c c i i c c c
Pred labels:  c c c c c c c i c c c c c c c c c c c c c i c c c
Gold:         Mi immaginerei un corso in un gruppo numericamente piccolo - 4 persone al massimo - se questo non è possibile prenderò delle lezioni individuali .
Correction 1: Mi immaginerei un corso in un gruppo nuovamente piccolo - 4 persone al massimo - se questo non è possibile prenderai delle lezioni individuali .
Correction 2: Mi immaginerei un corso in un gruppo nuovamente piccolo - 4 persone al massimo - se questo non è possibile prenderai dei lezioni individuali .
Correction 3: Mi immaginerei un corso in un gruppo nuovamente piccolo - 4 persone al massimo - se questo non è possibile prenderesti delle lezioni individuali .
Correction 4: Mi immaginerei un corso in un gruppo nuovamente piccolo - 4 pers

 25%|███████████▎                                 | 2/8 [01:18<03:52, 38.77s/it]


Original:     Secondo me i tuoi amici perdonano e non ti chiedono più perchè non ti faccio più sentire .
Gold labels:  c c c c c i c c c c c i c c i c c c
Pred labels:  c c c c c c c c c c c i c c c c c c
Gold:         Secondo me i tuoi amici perdonano e non ti chiedono più perché non ti fai più sentire .
Correction 1: Secondo me i tuoi amici perdonano e non ti chiedono più perché non ti faccio più sentire .
Correction 2: Secondo me i tuoi amici perdonano e non ti chiedono più il motivo per cui non ti faccio più sentire .
Correction 3: Secondo me i tuoi amici perdonano e non ti chiedono più il motivo per cui non ti faccio più sentirvi .
Correction 4: Secondo me i tuoi amici perdonano e non ti chiedono più il motivo per cui non ti faccio più sentirti .
Correction 5: Secondo me i tuoi amici perdonano e non ti chiedono più il motivo per cui non ti faccio più sentire .
Error types:  ['O_Graph', 'G_Agr', 'G_Valency']



 38%|████████████████▉                            | 3/8 [02:04<03:31, 42.25s/it]

...


Original:     Anche io ho una notizia a darvi , sono fidanzata di un ragazzo gentili e mi piacerebbe molto andare al vostro matrimonio con lui così potresti conoscerlo .
Gold labels:  c c c c c i c c c c i c c i c c c c c c c c c c c i c c
Pred labels:  c c c c c i c c c c c c c c c c c c c c c c c c c c c c
Gold:         Anche io ho una notizia da darvi , sono fidanzata con un ragazzo gentile e mi piacerebbe molto andare al vostro matrimonio con lui così potreste conoscerlo .
Correction 1: Anche io ho una notizia a darvi, sono fidanzata con un ragazzo gentile e mi piacerebbe molto andare al vostro matrimonio con lui così potresti conoscerlo .
Correction 2: Anche io ho una notizia a darvi, sono fidanzata di un ragazzo gentile e mi piacerebbe molto andare al vostro matrimonio con lui e così potresti conoscerlo .
Correction 3: Anche io ho una notizia a darvi, sono fidanzata con un ragazzo gentile e mi piacerebbe molto andare al vostro matrimonio con lui e così potresti conoscerlo .
Corr

 88%|██████████████████████████████████████▌     | 7/8 [42:05<09:18, 558.86s/it]


Original:     Probabilmente arrivò il 18 novembre e resterò fino al 22 novembre .
Gold labels:  c i c c c c c c c c c c
Pred labels:  c c c c c c c c c c c c
Gold:         Probabilmente arriverò il 18 novembre e resterò fino al 22 novembre .
Correction 1: Probabilmente arrivò il 18 novembre e resterò fino al 22 novembre.
Correction 2: Probabilmente arriverà il 18 novembre e resterò fino al 22 novembre.
Correction 3: Probabilmente arrivò il 18 novembre e resterò fino al 23 novembre .
Correction 4: Probabilmente arrivò il 18 novembre e rimarrò fino al 22 novembre .
Correction 5: Probabilmente arrivò il 18 novembre e resterò fino al 21 novembre .
Error types:  ['G_Agr']



100%|████████████████████████████████████████████| 8/8 [43:43<00:00, 327.90s/it]


Original:     Invece , l' inglese , che lo aveva usato prima moltissimo quando lavorava , solo l' uso quando stiamo con gli amici stranieri che non parlano italiano .
Gold labels:  c i c c c c i i c i c i i c i i i c c c c c c c c c c c
Pred labels:  c c c c c c c c c c c c c c c c c c c c c c c c c c c c
Gold:         Invece l' inglese , che ho usato moltissimo quando lavoravo , solo quando stiamo con gli amici stranieri che non parlano italiano .
Correction 1: Invece, l'inglese, che lo aveva usato molto prima quando lavorava, lo usiamo solo quando stiamo con gli amici stranieri che non parlano italiano .
Correction 2: Invece, l'inglese, che lo aveva usato molto prima quando lavorava, l'uso solo quando stiamo con gli amici stranieri che non parlano italiano .
Correction 3: Invece, l'inglese, che lo aveva usato molto prima quando lavorava, lo uso solo quando stiamo con gli amici stranieri che non parlano italiano .
Correction 4: Invece, l'inglese, che lo aveva usato molto prima quando

## False Positives and False Negatives

### ROBERTA --> G_Wo

In [43]:
all_G_Wo_roberta_corrections_FN_FP = []

for sentence in tqdm(roberta_merlin_fn_fp_error_type['G_Wo']):
    sentence_corrections = get_corrections_tokenized_with_labels_from_data(sentence, client)
    all_G_Wo_roberta_corrections_FN_FP.append(sentence_corrections)

100%|█████████████████████████████████████████████| 1/1 [01:33<00:00, 93.24s/it]


Original:     Veramente spero che , con l' impegno messo ad imparare bene l' italiano , adesso sia il momento per me di poter trovare un bel' lavoro .
Gold labels:  i i i c c c c c c c c c c c c c c c c c c c c c i c c
Pred labels:  c c c c c c c c i c c c c c c c c c c c c c c c i c c
Gold:         Spero che , con l' impegno messo ad imparare bene l' italiano , adesso sia il momento per me di poter trovare un bel lavoro .
Correction 1: Veramente spero che, con l'impegno messo ad imparare bene l'italiano, adesso sia il momento per me di poter trovare un bel lavoro .
Correction 2: Veramente spero che, con l'impegno messo a imparare bene l'italiano, adesso sia il momento per me di poter trovare un bel lavoro .
Correction 3: Veramente spero che, con l'impegno messo ad imparare bene italiano, adesso sia il momento per me di poter trovare un bel lavoro .
Correction 4: Veramente spero che, con l'impegno messo a imparare bene italiano, adesso sia il momento per me di poter trovare un bel lav

### ROBERTA --> G_Verb

In [45]:
all_G_Verb_roberta_corrections_FN_FP = []

for sentence in tqdm(roberta_merlin_fn_fp_error_type['G_Verb']):
    sentence_corrections = get_corrections_tokenized_with_labels_from_data(sentence, client)
    all_G_Verb_roberta_corrections_FN_FP.append(sentence_corrections)

 20%|█████████                                    | 1/5 [00:47<03:09, 47.37s/it]


Original:     Adesso non credete che è anche in questo corso !
Gold labels:  c c i i i c c c c c
Pred labels:  c c c c c c i c c c
Gold:         Adesso non crederete chi è anche in questo corso !
Correction 1: Adesso non credete che sia anche in questo corso!
Correction 2: Adesso non credete che è presente anche in questo corso!
Correction 3: Adesso non credete che è nel corso anche!
Correction 4: Adesso non credete che è anche in questo corso?
Correction 5: Adesso non credete che è nella formazione anche!
Correction 6: (Note: The fifth correction is a variation of the second, as there are only four unique corrections possible given the constraints.)
Error types:  ['G_Verb', 'O_Graph', 'G_Valency']



 40%|██████████████████                           | 2/5 [02:42<04:21, 87.22s/it]


Original:     Adesso Vi prego di riflettere in che modo dovete migliorare la Vostra programma nel """" Residence del Lago """" o di corretare la pubblicità che poi tutti possono vedere la realtà .
Gold labels:  c c c c c c c c c c i i c c c c c c c c c i c c i c c i c c c c
Pred labels:  c c c c c c c c c c i c i c c c c c c c c i c c c c c c c c c c
Gold:         Adesso Vi prego di riflettere in che modo dovete migliorare il Vostro programma nel """" Residence del Lago """" o di correggere la pubblicità che poi tutti possano vedere la realtà .
Correction 1: Adesso vi prego di riflettere in che modo dovete migliorare la vostra programma nel "Residence del Lago" o di corretta la pubblicità che poi tutti possano vedere la realtà .
Correction 2: Adesso vi prego di riflettere in che modo dovete migliorare la vostra programma nel "Residence del Lago" o di correggere la pubblicità che poi tutti possano vedere la realtà .
Correction 3: Adesso vi prego di riflettere in che modo dovete miglior

 60%|███████████████████████████                  | 3/5 [03:04<01:55, 57.54s/it]


Original:     Ti scritto qualcosa ti me .
Gold labels:  c i c i c c
Pred labels:  c c c i i c
Gold:         Ti scrivo qualcosa di me .
Correction 1: Ti scrivo qualcosa ti me .
Correction 2: Ti scrivo qualcosa tu me .
Correction 3: Ti scrivi qualcosa ti me .
Correction 4: Ti scrivi qualcosa tu me .
Correction 5: Ti scriverai qualcosa ti me .
Error types:  ['G_Pos', 'G_Verb']



 80%|████████████████████████████████████         | 4/5 [04:28<01:07, 67.98s/it]


Original:     Io vi ringrazio di avermi invitare , però non potrei venire perché devo andare in Francia per vedere la mia famiglia , che no visto di dopo le vacanzia di natal .
Gold labels:  c c c c c i c c c i c c c c c c c c c c c i c i i i c c i c i c
Pred labels:  c c c c c i c c c c c c c c c c c c c c c c c i i i c i i c i c
Gold:         Io vi ringrazio di avermi invitata , però non posso venire perché devo andare in Francia per vedere la mia famiglia che non vedo da dopo le vacanze di Natale .
Correction 1: Io vi ringrazio di avermi invitato, però non potrei venire perché devo andare in Francia per vedere la mia famiglia, che non ho visto dopo le vacanze di Natale.
Correction 2: Io vi ringrazio di avermi invitato, ma non posso venire perché devo andare in Francia per vedere la mia famiglia, che non ho visto dopo le vacanze di Natale.
Correction 3: Io vi ringrazio di avermi invitato, tuttavia non potrei venire perché devo andare in Francia per vedere la mia famiglia, che non ho

100%|█████████████████████████████████████████████| 5/5 [06:20<00:00, 76.10s/it]


Original:     Se si cercava un' alternativa , non si è trovata ciò che i campi da tennis sono stati in riparazione e l' animazione c' era solo una sera la settimana .
Gold labels:  c c c c c c c c i i c i c c c c i i c c c c c c c c c c c c c
Pred labels:  c c c c c c c c i i i c c c c c i c c c c c c c c c c c c c c
Gold:         Se si cercava un' alternativa , non si trovava ciò perché i campi da tennis erano in riparazione e l' animazione c' era solo una sera la settimana .
Correction 1: Se si cercava un'alternativa, non si è trovata. Cosa i campi da tennis sono stati in riparazione e l'animazione c'era solo una sera la settimana.
Correction 2: Se si cercava un'alternativa, non si è trovata. Ciò che i campi da tennis sono stati in riparazione e l'animazione c'era solo una sera la settimana.
Correction 3: Se si cercava un'alternativa, non si è trovata ciò che i campi da tennis sono stati in riparazione. E l'animazione c'era solo una sera la settimana.
Correction 4: Se si cercava un'

### ROBERTA --> G_Art

In [47]:
all_G_Art_roberta_corrections_FN_FP = []

for sentence in tqdm(roberta_merlin_fn_fp_error_type['G_Art']):
    sentence_corrections = get_corrections_tokenized_with_labels_from_data(sentence, client)
    all_G_Art_roberta_corrections_FN_FP.append(sentence_corrections)

 12%|█████▋                                       | 1/8 [01:14<08:38, 74.08s/it]


Original:     La sera io posso andare al un concerto con la mia padre o andare in un ristorante con la mia famiglia .
Gold labels:  c c c c c c i c c i i c c c c c c c c c c c
Pred labels:  c c c c c i c c c i i c c c c c c c c c c c
Gold:         La sera io posso andare al concerto con mio padre o andare in un ristorante con la mia famiglia .
Correction 1: La sera io posso andare a un concerto con la mia madre o andare in un ristorante con la mia famiglia .
Correction 2: La sera io posso andare al concerto con la mia sorella o andare in un ristorante con la mia famiglia .
Correction 3: La sera io posso andare al concerto con il mio amico o andare in un ristorante con la mia famiglia .
Correction 4: La sera io posso andare a un concerto con la mia moglie o andare in un ristorante con la mia famiglia .
Correction 5: La sera io posso andare a un concerto con il mio cugino o andare in un ristorante con la mia famiglia .
Error types:  ['G_Morphol_Wrong', 'G_Art']



 25%|███████████▎                                 | 2/8 [01:58<05:41, 56.92s/it]


Original:     E se hai usato Facebook per fare le conoscenze , poi provare anche Google + .
Gold labels:  c c c c c c c i c c i c c i i c
Pred labels:  c c c c c c c c c c i i c c c c
Gold:         E se hai usato Facebook per fare conoscenze , puoi provare anche Google+ .
Correction 1: E se hai usato Facebook per fare conoscenze, poi provare anche Google+.
Correction 2: E se hai usato Facebook per fare le conoscenze, poi provare anche Google.
Correction 3: E se hai usato Facebook per fare le conoscenze, poi provare Google+.
Correction 4: E se hai usato Facebook per fare le conoscenze, poi prova anche Google+.
Correction 5: E se hai usato Facebook per fare le conoscenze, poi provare anche Google.
Error types:  ['G_Inflect_Inexist', 'G_Art', 'NO_LABEL']



 38%|████████████████▉                            | 3/8 [04:21<07:58, 95.80s/it]

...


Original:     Penso che la societa sia formata così , anch' io avevo quel pensiero ma quando sono andato in viaggio in Africa , mi sono riso conto che siamo privilegiati , tutti noi che viviamo in uno stato senza dittatura e dove esiste una copertura malattia sociale .
Gold labels:  c c c i c c c c i c c c c c c c c c c c c c c c i c c c c c c c c c c c c c c c c c c c i i c
Pred labels:  c c c i c i c c c c c c c c c c c c c c c c c c i c c c c c c c c c c c c c c c c c c c i c c
Gold:         Penso che la società sia formata così , Anch' io avevo quel pensiero ma quando sono andato in viaggio in Africa , mi sono reso conto che siamo privilegiati , tutti noi che viviamo in uno stato senza dittatura e dove esiste una copertura malattie .
Correction 1: Penso che la società sia formata così, anch'io avevo quel pensiero ma quando sono andato in viaggio in Africa, mi sono reso conto che siamo privilegiati, tutti noi che viviamo in uno stato senza dittatura e dove esiste una copertura mala

 88%|███████████████████████████████████████▍     | 7/8 [09:54<01:19, 79.12s/it]


Original:     Quanto costa il questo viaggio ?
Gold labels:  c c c i c c
Pred labels:  c c i c c c
Gold:         Quanto costa il viaggio ?
Correction 1: Quanto costa questo viaggio ?
Correction 2: Quanto costa il viaggio ?
Correction 3: Quanto costa questo?
Correction 4: Quanto questo viaggio costa ?
Correction 5: Quanto il viaggio costa ?
Error types:  ['G_Art']



100%|█████████████████████████████████████████████| 8/8 [10:20<00:00, 77.58s/it]


Original:     Conosco un veramente buonissimo !
Gold labels:  i i c c c
Pred labels:  c c i c c
Gold:         conosco uno veramente buonissimo !
Correction 1: Conosco un buonissimo!
Correction 2: Conosco un veramente buono!
Correction 3: Conosco un veramente buonissimo.
Correction 4: Conosco un molto buonissimo!
Correction 5: Conosco un davvero buonissimo!
Error types:  ['G_Art', 'G_Valency']



### ROBERTA --> G_Valency

In [49]:
all_G_Valency_roberta_corrections_FN_FP = []

for sentence in tqdm(roberta_merlin_fn_fp_error_type['G_Valency']):
    sentence_corrections = get_corrections_tokenized_with_labels_from_data(sentence, client)
    all_G_Valency_roberta_corrections_FN_FP.append(sentence_corrections)

 20%|█████████                                    | 1/5 [00:28<01:55, 28.96s/it]


Original:     Adesso non credete che è anche in questo corso !
Gold labels:  c c i i i c c c c c
Pred labels:  c c c c c c i c c c
Gold:         Adesso non crederete chi è anche in questo corso !
Correction 1: Adesso non credete che sia anche in questo corso!
Correction 2: Adesso non credete che è anche nel corso!
Correction 3: Adesso credete che è anche in questo corso?
Correction 4: Adesso non credevo che era anche in questo corso!
Error types:  ['G_Verb', 'O_Graph', 'G_Valency']



 40%|██████████████████                           | 2/5 [01:07<01:43, 34.44s/it]


Original:     Quando ti vuoi inizarer il nuovo posto di lavaro ?
Gold labels:  c i c i c c c c i c
Pred labels:  c c c i i c c c i c
Gold:         Quando vuoi iniziare il nuovo posto di lavoro ?
Correction 1: Quando ti voglio iniziare il nuovo posto di lavoro ?
Correction 2: Quando ti vuoi iniziare il nuovo posto di lavoro ?
Correction 3: Quando ti vuoi iniziarizzare il nuovo posto di lavoro ?
Correction 4: Quando ti vuoi iniziare il nuovo posti di lavore ?
Correction 5: Quando ti vuoi iniziare il nuovo posto di lavoro ?
Error types:  ['O_Graph', 'G_Valency']



 60%|███████████████████████████                  | 3/5 [01:29<00:57, 28.82s/it]


Original:     Io do una mano con tuo curriculum .
Gold labels:  c i c c c i c c
Pred labels:  c c c c i i c c
Gold:         Io do una mano con tuo curriculum .
Correction 1: Io do una mano con il tuo curriculum .
Correction 2: Io aiuto con il tuo curriculum .
Correction 3: Io contribuisco al tuo curriculum .
Correction 4: Con il tuo curriculum io aiuto .
Correction 5: Con il tuo curriculum io contribuisco .
Error types:  ['G_Art', 'G_Valency']



 80%|████████████████████████████████████         | 4/5 [02:10<00:33, 33.82s/it]


Original:     Sa-unreadable-ete essatamente che voi non avete ancora .
Gold labels:  i i i c c c c c
Pred labels:  i i c c c i c c
Gold:         unreadable esattamente che voi non avete ancora .
Correction 1: Sa uno siete esattamente che voi non avete ancora .
Correction 2: Sa un unreadable e siete esattamente che voi non avete ancora .
Correction 3: Sa un ette siete esattamente che voi non avete ancora .
Correction 4: Sa un eta siete esattamente che voi non avete ancora .
Correction 5: Sa un ete siete esattamente che voi non avete ancora .
Error types:  ['O_Graph', 'G_Valency']



100%|█████████████████████████████████████████████| 5/5 [02:39<00:00, 31.92s/it]


Original:     Conosco un veramente buonissimo !
Gold labels:  i i c c c
Pred labels:  c c i c c
Gold:         conosco uno veramente buonissimo !
Correction 1: Conosco un buonissimo!
Correction 2: Conosco un veramente buono!
Correction 3: Conosco un buonissimo restaurant!
Correction 4: Conosco un vero e proprio buonissimo!
Correction 5: Conosco un veramente buonissimo.
Error types:  ['G_Art', 'G_Valency']



### ROBERTA --> G_Agr

In [51]:
all_G_Agr_roberta_corrections_FN_FP = []

for sentence in tqdm(roberta_merlin_fn_fp_error_type['G_Agr']):
    sentence_corrections = get_corrections_tokenized_with_labels_from_data(sentence, client)
    all_G_Agr_roberta_corrections_FN_FP.append(sentence_corrections)

100%|█████████████████████████████████████████████| 1/1 [00:32<00:00, 32.02s/it]


Original:     Ma tutto era al contrario come era scritta nel Suo annuncio .
Gold labels:  i c c i c i c i c c c c
Pred labels:  c c c c c c i c c c c c
Gold:         ma tutto era il contrario come era scritto nel Suo annuncio .
Correction 1: Ma tutto era al contrario come era scritto nel Suo annuncio .
Correction 2: Ma tutto era al contrario come era scritto nel suo annuncio .
Correction 3: Ma tutto era contrario come era scritto nel Suo annuncio .
Correction 4: Ma tutto era contrario come era scritto nel suo annuncio .
Error types:  ['G_Prep', 'G_Agr', 'NO_LABEL']



# GENERATION FOR FP ONLY - MERLIN - ROBERTA

In [55]:
all_false_positive_roberta = []

for sentence in tqdm(roberta_merlin_only_fp):
    sentence_corrections = get_corrections_tokenized_with_labels_from_data(sentence, client)
    all_false_positive_roberta.append(sentence_corrections)

  2%|█                                           | 1/42 [00:24<16:40, 24.41s/it]


Original:     Ciao Caro , come stai ?
Gold labels:  c c c c c c
Pred labels:  c i c c c c
Gold:         Ciao Caro , come stai ?
Correction 1: Ciao caro, come stai?
Correction 2: Ciao cara, come stai?
Correction 3: Ciao caro, come stai?
Correction 4: Ciao caro, come stai.
Correction 5: Ciao caro, come stai.
Error types:  []



  5%|██                                          | 2/42 [01:01<21:03, 31.58s/it]


Original:     Tanto non bisognò di prenotare un alloggio .
Gold labels:  c c i c c c c c
Pred labels:  c c i i c c c c
Gold:         Tanto non bisogno di prenotare un alloggio .
Correction 1: Tanto non bisogna prenotare un alloggio .
Correction 2: Tanto non fu necessario prenotare un alloggio .
Correction 3: Tanto non si dovette prenotare un alloggio .
Correction 4: Tanto non era necessario prenotare un alloggio .
Correction 5: Tanto non bisognò prenotare un alloggio .
Error types:  ['G_Verb_main']



  7%|███▏                                        | 3/42 [01:07<13:00, 20.00s/it]

...


Original:     Il mio albergho à vicino alla Monumento X .
Gold labels:  c c i i c c c c c
Pred labels:  c c i i c i c c c
Gold:         Il mio albergo è vicino alla Monumento X .
Correction 1: Il mio albergo è vicino al Monumento X .
Correction 2: Il mio albergo à vicino al Monumento X .
Correction 3: Il mio albergho è vicino a Monumento X .
Correction 4: Il mio albergho à vicino a Monumento X .
Correction 5: Il mio albergo è vicino alla Monumento X .
Error types:  ['H_Intelltxt']



 98%|█████████████████████████████████████████▉ | 41/42 [33:36<01:01, 61.35s/it]


Original:     Poi scrivere una cartolina anche o puoi chiamiami a questo numero : 0345-11111111 !
Gold labels:  i c c c i c c i c c c c c c
Pred labels:  i c c c i c c i i c c c c c
Gold:         Puoi scrivere una cartolina o puoi chiamarmi a questo numero : 0345-11111111 !
Correction 1: Poi scrivere una cartolina anche oppure puoi chiamami a questo numero: 0345-11111111!
Correction 2: Poi scrivere una cartolina anche o puoi chiamami a questo numero: 0345-11111111!
Correction 3: Poi scrivere una cartolina anche oppure puoi chiamarmi a questo numero: 0345-11111111!
Correction 4: Poi scrivere una cartolina anche o puoi chiamarmi a questo numero: 0345-11111111!
Correction 5: Poi scrivere una cartolina anche oppure puoi chiamarmi a questo numero: 0345-11111111!
Error types:  ['G_Pos', 'G_Conj', 'O_Graph']



100%|███████████████████████████████████████████| 42/42 [35:19<00:00, 50.47s/it]


Original:     Crispi 119 80127 Caserta Oggetto : La Sua lettera Caro Giorgio , in riferimento della Sua lettera Le scrivo .
Gold labels:  c c c c c c c c c c c c c c i c c c c c
Pred labels:  c c c c c c i c c c c c c c i c c i c c
Gold:         Crispi 119 80127 Caserta Oggetto : La Sua lettera Caro Giorgio , in riferimento alla Sua lettera Le scrivo .
Correction 1: Crispi 119 80127 Caserta Oggetto: La Sua lettera Caro Giorgio, in riferimento alla Sua lettera Le scrivo.
Correction 2: Crispi 119 80127 Caserta Oggetto: La Sua lettera Caro Giorgio, in riferimento a Sua lettera Le scrivo.
Correction 3: Crispi 119 80127 Caserta Oggetto: La Sua lettera Caro Giorgio, in riferimento alla Sua Le scrivo.
Correction 4: Crispi 119 80127 Caserta Oggetto: La Sua lettera Caro Giorgio, in riferimento alla Sua lettera .
Correction 5: Crispi 119 80127 Caserta Oggetto: La Sua lettera Caro Giorgio, in riferimento a Sua lettera .
Error types:  ['NO_LABEL']

